In [1]:
import pandas as pd
import os
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=ebc24c7f5b1f5e94d3b4c3b2b74f0df0d7f1caaf24f1a10fdf732c33abac09d1
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [2]:
#code to work with data in google collab
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive"

Mounted at /content/drive
/content/drive/MyDrive


In [38]:
#read file 
df = pd.read_parquet("dutch-restaurant-reviews.parquet")

In [39]:
#remove NA values for rating score and convert to float values 
df = df[~df['reviewScoreOverall'].isna()]

In [40]:
print(len(df))
print(df["restoId"].nunique())
df = df[["reviewText", "reviewScoreOverall"]]

368529
12234


In [41]:
#add review_length column to determine amount of words in each review
df["review_length"]= df["reviewText"].str.split().apply(len)

In [42]:
#delete reviews with only one word, pending reviews and with more than 512 words
df_1 = df[df["review_length"] != 1]
df_2 = df_1[df_1["reviewText"]!= "- Recensie is momenteel in behandeling -"]
df_3 = df_2[df_2["review_length"] <= 512]
print(len(df_3))

365038


In [45]:
#delete reviews that do not contain any letter
import re
df_3['letter_first'] = df_3["reviewText"].apply(str.lower)


df_3["letter"] = df_3["letter_first"].apply(str.islower)

df_3["letter"].value_counts()
df_4 = df_3[df_3["letter"] == True]


<ipython-input-45-90de28d5e29c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['letter_first'] = df_3["reviewText"].apply(str.lower)
<ipython-input-45-90de28d5e29c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3["letter"] = df_3["letter_first"].apply(str.islower)


In [46]:
#add languagereview column for identification of language of review text and delete non-Dutch reviews
from langdetect import detect
for index, row in df_4['reviewText'].items():
    lang = detect(row) #detecting each row
    df_4.loc[index, 'Languagereview'] = lang

#delete non-Dutch reviews 
df_5 = df_4[df_4["Languagereview"] == "nl"]

<ipython-input-46-6af34d900f9a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_4.loc[index, 'Languagereview'] = lang


In [48]:

path = '/content/drive/MyDrive/cleaned_data.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_5.to_csv(f)